In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("./data/gemstone_train.csv")

In [3]:
df.drop(labels=["id"],axis=1,inplace=True)

In [4]:
df.tail()

,carat,cut,color,clarity,depth,table,x,y,z,price
193568,0.31,Ideal,D,VVS2,61.1,56.0,4.35,4.39,2.67,1130
193569,0.70,Premium,G,VVS2,60.3,58.0,5.75,5.77,3.47,2874
193570,0.73,Very Good,F,SI1,63.1,57.0,5.72,5.75,3.62,3036
193571,0.34,Very Good,D,SI1,62.9,55.0,4.45,4.49,2.81,681
193572,0.71,Good,E,SI2,60.8,64.0,5.73,5.71,3.48,2258


In [5]:
X = df.drop(labels=["price"],axis=1)

In [6]:
y = df[["price"]]

In [7]:
y

,price
0,13619
1,13387
2,2772
3,666
4,14453
...,...
193568,1130
193569,2874
193570,3036
193571,681


In [8]:
categorical_cols = X.select_dtypes(include='object').columns

In [9]:
categorical_cols

Index(['cut', 'color', 'clarity'], dtype='object')

In [10]:
numerical_cols=X.select_dtypes(exclude='object').columns

In [11]:
numerical_cols

Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')

In [12]:
# Define the custom ranking for each ordinal variable
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [13]:
from sklearn.impute import SimpleImputer ## HAndling Missing Values
from sklearn.preprocessing import StandardScaler # HAndling Feature Scaling
from sklearn.preprocessing import OrdinalEncoder # Ordinal Encoding
## pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [14]:
#create numerical pipeline with object of pipeline 

num_pipeline = Pipeline(

    steps =[
        ('imputer', SimpleImputer()),
        ('scaler', StandardScaler())
    ]

)

In [15]:
#create cat pipeline with object of pipeline 
cat_pipeline = Pipeline (
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')), #inside cat we need mode as most frq value
        ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])) #passed cat from above 
    ]
)

In [16]:
#transforming 
preprocessor=ColumnTransformer(
    [
        
        ('num_pipeline',num_pipeline,numerical_cols), #operation numerical col as its float
        ('cat_pipeline',cat_pipeline,categorical_cols) 
    ]
)

In [17]:
## Train test split

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=30)

In [18]:
preprocessor.fit_transform(X_train)

array([[-0.97543926, -0.84960654, -0.12153081, ...,  4.        ,
         5.        ,  5.        ],
       [ 0.2351953 ,  1.83363716, -0.12153081, ...,  1.        ,
         1.        ,  2.        ],
       [ 0.49461699,  0.81585507,  0.39980029, ...,  3.        ,
         3.        ,  4.        ],
       ...,
       [ 0.45138004,  1.55606023, -0.6428619 , ...,  1.        ,
         3.        ,  2.        ],
       [ 0.66756478, -1.77486298,  1.44246248, ...,  4.        ,
         3.        ,  4.        ],
       [ 0.25681377,  0.81585507, -0.12153081, ...,  4.        ,
         3.        ,  2.        ]])

In [19]:
#preprocessor.transform(X_test)
#only transform --> collected in entire sort

In [20]:
#get all feature name 
preprocessor.get_feature_names_out()

array(['num_pipeline__carat', 'num_pipeline__depth',
       'num_pipeline__table', 'num_pipeline__x', 'num_pipeline__y',
       'num_pipeline__z', 'cat_pipeline__cut', 'cat_pipeline__color',
       'cat_pipeline__clarity'], dtype=object)

In [21]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [22]:
print(X_train.columns)

Index(['num_pipeline__carat', 'num_pipeline__depth', 'num_pipeline__table',
       'num_pipeline__x', 'num_pipeline__y', 'num_pipeline__z',
       'cat_pipeline__cut', 'cat_pipeline__color', 'cat_pipeline__clarity'],
      dtype='object')


In [23]:
"""linear regression
ridge regression
lasso regression
elastic net"""

'linear regression\nridge regression\nlasso regression\nelastic net'

In [24]:
## Model Training

from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [25]:
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [26]:
## Train multiple models

models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}

In [27]:
trained_model_list=[]
model_list=[]
r2_list=[]

In [28]:
list(models)

['LinearRegression', 'Lasso', 'Ridge', 'Elasticnet']

In [29]:
for i in range(len(list(models))):
    model=list(models.values())[i]
    print(model)

LinearRegression()
Lasso()
Ridge()
ElasticNet()


In [30]:
models.keys()

dict_keys(['LinearRegression', 'Lasso', 'Ridge', 'Elasticnet'])

In [31]:
models.values()

dict_values([LinearRegression(), Lasso(), Ridge(), ElasticNet()])

In [32]:
for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    #this is a validation(test) score
    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')

LinearRegression
Model Training Performance
RMSE: 1013.9047094344003
MAE: 674.0255115796847
R2 score 93.68908248567512


Lasso
Model Training Performance
RMSE: 1013.8723151049944
MAE: 675.2036880701619
R2 score 93.68948574778251


Ridge
Model Training Performance
RMSE: 1013.9058997761106
MAE: 674.0565132295964
R2 score 93.68906766741323


Elasticnet
Model Training Performance
RMSE: 1513.914035022976
MAE: 1050.7468664314322
R2 score 85.92978759337907




In [33]:
trained_model_list

[]

In [34]:
model_list

['LinearRegression', 'Lasso', 'Ridge', 'Elasticnet']

In [35]:
r2_list

[0.9368908248567511,
 0.9368948574778251,
 0.9368906766741323,
 0.8592978759337908]